In [ ]:
! pip install sklearn

# Imports

In [1]:
from fastai.text import *

In [2]:
import sys

sys.path.append("../../")
from eval.exp.nb_evaluation import *

sys.path.append("../../../")
from src.proc.exp.nb_proc import *
from src.prep.exp.nb_prep import *

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
# setup paths and model type
model_path = Path("/tf/data/models")
data_path  = Path("/tf/data/datasets")

task_type = "merged"

# Load Data

In [6]:
df_trn, df_val, df_tst = read_data(data_path/task_type)

In [7]:
bs = 8

Percentage of data to be used: sample 

In [ ]:
data = gen_lm_data(df_trn, df_val, task_type, data_path, bs = bs)
data.save(task_type + '/data_lm_100pct.pkl')

In [8]:
data = load_data(data_path/task_type, 'data_lm_01pct.pkl', bs=bs)

In [9]:
len(data.train_ds), len(data.valid_ds)

(4929, 1054)

# Model Setup

In [11]:
pretrained = False

In [12]:
# amit experiments
learn = language_model_learner(data, Transformer, drop_mult = 0.1, pretrained = pretrained, metrics=[accuracy])

In [ ]:
learn.lr_find()
learn.recorder.plot()

# Model Training

In [ ]:
# Set hyperparameters
max_lr = 1e-2
moms = (0.5, .75)
pct_strt = 0.02
a_epochs = 10

In [ ]:
callback_fns = [
    callbacks.SaveModelCallback(
        learn, every='improvement',
        monitor='valid_loss', name='transformer_save_model'
    ),
    callbacks.EarlyStoppingCallback(
        learn, monitor='valid_loss', min_delta = 0.01,
        patience = 3
    )
]

In [ ]:
#amit experiments
learn.fit_one_cycle(a_epochs, max_lr, callbacks = callback_fns) #, moms=moms, pct_start = pct_strt)

In [ ]:
learn.save(model_path/'awd_lstm_do_30pct')

In [13]:
learn.load('transformer_save_model')

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (4929 items)
x: LMTextList
▁x x bo s ▁@ override ▁public ▁void ▁un register service ( service instance < t > ▁service ) ▁throws ▁x x ma j ▁exception ▁{ ▁entry < t > ▁entry ▁= ▁service s . remove ( service . get id ()); ▁internal un register service ( entry ); ▁}<$ comment $> un register ▁/ ▁remove ▁a ▁group ▁instance ▁@ param ▁service ▁the ▁group ▁@ throws ▁x x ma j ▁exception ▁errors,▁x x bo s ▁list < i _ cms simple context menu entry < set < string >>> ▁get menu entries () ▁{ ▁if ▁( m _ menu entries ▁== ▁null ) ▁{ ▁m _ menu entries ▁= ▁new ▁array list < i _ cms simple context menu entry < set < string >>> (); ▁m _ menu entries . add ( new ▁user entry ()); ▁m _ menu entries . add ( new ▁send broadcast entry ()); ▁m _ menu entries . add ( new ▁ kill entry ()); ▁} ▁return ▁m _ menu entries ; ▁}<$ comment $> return s ▁the ▁available ▁menu ▁entries . < p > ▁@ return ▁the ▁menu ▁entries,▁x x bo s ▁public ▁re vo ke ip rule s request ▁w

In [ ]:
learn.export(file = model_path/'awd_lstm_do_30pc.pkl')

In [ ]:
learn = load_model(df_trn, file = model_path/'awd_lstm_do_30pc.pkl')

In [ ]:
# learn.recorder.plot_losses()
figure_plot = learn.recorder.plot_losses(return_fig=True)
figure_plot.savefig(fname="transformer_xl_plot_losses.png", format='png')
from PIL import Image
Image.open('/tf/main/nbs/mdling/transformer_xl/transformer_xl_plot_losses.png')

# Model Evaluation

In [ ]:
print(df_val["query"][50000])
print("\n\n" + df_val["res"][50000])

In [ ]:
print(df_val["query"][75_000])
print("\n\n" + df_val["res"][75_000])

In [14]:
import sentencepiece as spm
sp = spm.SentencePieceProcessor()
sp.Load(str(data_path/"merged/model.model"))

True

In [ ]:
TEXT = df_val["query"][500]
N_WORDS = 200
N_SENTENCES = 2

In [ ]:
print(TEXT)
df_val["res"][500]

In [ ]:
print(sp.DecodePieces(learn.predict(TEXT, 100, temperature=0.75).split(" ")))

# Evaluation

In [ ]:
learn.predict??

In [15]:
# Grabs entire model's response up until special xxbos token,
# i.e. once model begins a new sentence we consider the model finished with its answer.
def get_res(mdl, inpt, n_toks = 1_000):
    res = mdl.predict(inpt, n_toks, temperature=0.75).split(" ")
    res = sp.DecodePieces(res).split(" ")
    try:
        end_res = res.index("xxbos")
    except:
        end_res = len(res) - 1
    
    res = " ".join(res[:end_res])[len(inpt.replace(" ", '')):]
    
    return res

In [16]:
res = get_res(learn, "public static void main() {return;}<$bug$>", n_toks = 10)
res # [0:len("public static void main() {return;}<$bug$>")]

'no'

In [50]:
#export
def eval_txt(mdl, ds):
    b1, b2, b3, b4 = [], [], [], []
    meteor = []
    rouge_l = []
    preds = []
    tokenizer = Tokenizer()
    for inpt, lbl in zip(ds["query"], ds["res"]):
        tok_len = len(sp.EncodeAsPieces(inpt))
        if tok_len > 1024:
            continue
            
        pred = get_res(mdl, inpt, n_toks = 600)
        
        tokens = tokenizer.process_all([lbl])
        print(tokens)
        lbl = ' '.join(tokens[0])
        print(lbl)
        preds.append(pred)
        # bleu 1-4
        b1.append(eval_bleu1([lbl], pred))
        b2.append(eval_bleu2([lbl], pred))
        b3.append(eval_bleu3([lbl], pred))
        b4.append(eval_bleu4([lbl], pred))
        
        # meteor
        meteor.append(eval_meteor([lbl], pred))
        
        # rouge
        rouge_l.append(eval_rougeL_single_ref([lbl], pred))
        
    return b1, b2, b3, b4, meteor, rouge_l, preds

In [38]:
tokenizer = Tokenizer()
tokens = tokenizer.process_all(df_val[-10:-9]['res'])
# tokens
' '.join(tokens[0])

'xxmaj marshall the given parameter object .'

In [ ]:
df_val.tail(10)

In [19]:
from statistics import mean, median, stdev

In [59]:
preds[2]

'reads a file using a given resource map. @paramarn @return a map containing all the directorys that are notified'

In [72]:
str(df_val['query'][len(df_val) - 8])

'public synchronized Map<String, MLArray> read(InputStream stream) throws IOException\n    {\n        return read(stream, new MatFileFilter());\n    }<$comment$>'

In [51]:
b1, b2, b3, b4, meteor, rouge_l, preds  = eval_txt(learn, df_val[-10:])

[['xxmaj', 'marshall', 'the', 'given', 'parameter', 'object', '.']]
xxmaj marshall the given parameter object .
[['xxmaj', 'gets', 'the', 'bridge', 'for', 'job', '.', '\n \n ', '@param', '_', 'jobname', 'the', '_', 'job', 'name', '\n ', '@param', '_', 'prune', 'the', '_', 'prune', '\n ', '@return', 'the', 'bridge4', 'job']]
xxmaj gets the bridge for job . 
 
  @param _ jobname the _ job name 
  @param _ prune the _ prune 
  @return the bridge4 job
[['xxmaj', 'reads', 'the', 'content', 'of', 'a', 'xxup', 'mat', '-', 'file', 'and', 'returns', 'the', 'mapped', 'content', '.', '\n ', '<', 'p', '>', '\n ', 'xxmaj', 'this', 'method', 'calls', '<', 'code', '>', 'read(stream', ',', 'new', 'matfilefilter', '(', ')', ')', '<', '/', 'code>.', '\n \n ', '@param', 'stream', '\n ', 'a', 'valid', 'xxup', 'mat', '-', 'file', 'stream', 'to', 'be', 'read', '\n ', '@return', 'the', 'same', 'as', '<', 'code>{@link', '#', 'getcontent', '(', ')', '}', '<', '/', 'code', '>', '\n ', '@throws', 'ioexception', 

In [43]:
preds

['marshall the given parameter object.',
 "method to kill the current sale pwstcommands @param sp_property xxmaj the name of the s thiscommand to continue @return the job's defaults map",
 'reads all properties from a doc. xxmaj the inputstream is <code>null< / code>. @param  xxmaj the input stream @return xxmaj the stream of attachment',
 'gets the given xxup eelement xxup cp nodeing structure. @param nodes the current treelist @return the first xxup nav node @throws kdfexception if not.',
 'removes the context to the list of calls to the list.',
 '{@inheritdoc}',
 "sets the field name for the field in the field. @param fieldname the name of the field when the field is set @param isdisplay true if the field can't be found, otherwise false",
 'transform a string to a string, call to xxup aws to read the content of the content, and returns it. xxmaj the {@link # ) method is called for {@code content} or {@link # format(int)}. @param id the common identifier of the content to be parsed. 

In [42]:
b1

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [54]:
mean(rougue)

0.179

In [27]:
df_val[-100:-95]

,query,res
105263,private String getProjectIdForProjectName(Stri...,Return the project id for a given project name...
105264,public File getFileFromDeletedFilesNotUpdated(...,Returns the name of the deleted file that is r...
105265,public JobInformationRecorder withBatchInforma...,Indicates the JobInformationRecorder to persis...
105266,public Object getAttribute(String name) {\r\n ...,返回这个窗口的私有属性或portal主控请求对象的共同属性\n\n@param name N...
105267,public VolumeStatusItem withEvents(VolumeStatu...,<p>\nA list of events associated with the volu...


In [56]:
preds[4]

'iteratively after a {@link managementeventstreammanager}'

In [ ]:
df_val[-10:]

In [ ]:
rouge_l[0]

In [55]:
mean(rouge_l[:][0]), mean(rouge_l[:][1]), mean(rouge_l[:][2])#, preds[:10]

(0.9277409729371751, 0.484819741733491, 0.2596623316452132)

In [ ]:
# Evaluation metrics for vulnerability detection - Accuracy, Precision, Recall
def eval_vuln(mdl, tst):
    tps, tns, fps, fns = 0, 0, 0, 0
    tot = 0
    for inpt, lbl in zip(tst["query"], tst["res"]):
        tok_len = len(sp.EncodeAsPieces(inpt))
        if tok_len > 1024:
#             print("Skipping because size is too big", tok_len)
            continue
        pred = get_res(mdl, inpt, n_toks = 10)
        if lbl == "yes":
            if pred == lbl:
                tps += 1
            else: fns += 1
        else:
            if pred == lbl:
                tns += 1
            else: fps += 1
                
        tot += 1
        torch.cuda.empty_cache()
            
    acc   = (tps + tns) / tot
    prec  = tps / (tps + fps) if (tps + fps) != 0 else 0.
    recal = tps / (tps + fns) if (tps + fns) != 0 else 0.
    
    return acc, prec, recal

In [ ]:
acc, prec, recal = eval_vuln(learn, df_val[:1000])

In [ ]:
acc, prec, recal

In [ ]:
len(df_val[:10])

In [ ]:
import torch
torch.cuda.empty_cache()